# Manipulation Modelling & Execution

The discussed components (KOMO, BotOp, NLP_Solver, RRT) provide basic ingredients for manipulation planning and execution. This tutorial is about how to practically use these in typical manipulation settings.

The first focus is on *manipulation modelling*. While KOMO provides a very powerful abstract framework to define all kinds of constraints, here we discuss what are concrete useful constraints for typical actions, e.g., picking and placing a box, or capsule. The *ManipulationModelling* class is meant to translate between typical actions and the abstract KOMO specification of the corresponding constraints.

The second focus is on the whole pipeline. We follow a basic sense-plan-act pipeline (not yet a fully integrated reactive framework such as SecMPC). To be more detailed, we assume the following basic steps in each loop:
* Perception: Update the configuration model to be in sync with the real world - using perception.
* Discrete decisions (task planning): Decide on discrete next actions, such as which object to pick or place next.
* Waypoint planning: Model the manipulation constraints for the next few actions and solve them to get a plan for the next few waypoints.
* Path planning: Create a fine-grained path/trajectory between waypoints, sometimes justing quick interpolation & optimization, sometimes using full fledge path finding (bi-directional RRT).
* Execution: Sending the path to BotOp for running it on the real system.

We neglect perception and discrete decision making here.

## Manipulation Modelling

We start with discussing manipulation modelling for standard box/cylinder grasping and placing.

In [1]:
import robotic as ry
import numpy as np
import random
import time

# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip

A basic configuration with a box and cylinder:

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.06,.06,.005]) \
    .setColor([1,.5,0]) \
    .setContact(1)

C.addFrame("capsule") \
    .setShape(ry.ST.capsule, [.2,.02]) \
    .setPosition([.25,.1,1.]) \
    .setColor([1,.5,0]) \
    .setContact(1)

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

Look into the definition of *ManipulationModelling* class! You see that this class provides wrapper methods to setup a komo problem. The following demonstrate the methods provided to model box and cylinder grasping IK problems:

### Box centered top grasp
There are 6 possible orientation of an orthonormal centered box grasp. Have a look at the `grasp_top_box` method!

In [3]:
C.setJointState(qHome)
for orientation in ['xy', 'xz', 'yx', 'yz', 'zx', 'zy']: #loops over the 6 possible grasps
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    man.grasp_top_box(1., gripper, box, orientation)
    
    # solve it
    pose = man.solve()
    
    # check feasibility and display
    if man.ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_top_box with orientation {orientation}\nret: {man.ret}')
    else:
        print(' -- infeasible')

  -- feasible:
     { time: 0.0185, evals: 45, done: 1, feasible: 1, sos: 0.121124, f: 0, ineq: 0, eq: 0.0594381 }
  -- feasible:
     { time: 0.005344, evals: 18, done: 1, feasible: 1, sos: 0.0353606, f: 0, ineq: 0, eq: 0.00981419 }
  -- feasible:
     { time: 0.006397, evals: 22, done: 1, feasible: 1, sos: 0.0508421, f: 0, ineq: 0, eq: 0.00981408 }
  -- feasible:
     { time: 0.0034, evals: 12, done: 1, feasible: 1, sos: 0.00573579, f: 0, ineq: 0, eq: 0.00982528 }
  -- feasible:
     { time: 0.00447, evals: 15, done: 1, feasible: 1, sos: 0.0401536, f: 0, ineq: 0, eq: 0.00981435 }
  -- feasible:
     { time: 0.003661, evals: 13, done: 1, feasible: 1, sos: 0.0279681, f: 0, ineq: 0, eq: 0.00979114 }


### Box general grasp
We do not have to grasp a box in the center or orthonormally. We can only specify along which axis the fingers should press, and that they need to be inside a margin of the box sides.  Have a look at the `grasp_box` method! To illustrate the gained degrees of freedom, we also impose a random bias (leading to different solutions in nullspace):

In [4]:
C.setJointState(qHome)
limits = C.getJointLimits()
for orientation in ['x', 'y', 'z']:
    for i in range(10):
        # setup the manipulation problem
        man = manip.ManipulationModelling(C)
        man.setup_inverse_kinematics()
        # ... with random bias in joint space
        qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
        man.bias(1., qBias, 1e0)
        # ... and general, non-centered box grasping
        man.grasp_box(1., gripper, box, palm, orientation, margin=.02)
        
        # solve
        pose = man.solve()
        
        # if feasible, display
        if man.ret.feasible:
            C.setJointState(pose[0])
            C.view(True, f'grasp_box with orientation {orientation}\nret: {man.ret}')
        else:
            print('-- infeasible', i, orientation)

  -- feasible:
     { time: 0.011672, evals: 64, done: 1, feasible: 1, sos: 7.61771, f: 0, ineq: 8.72807e-07, eq: 5.06128e-06 }
  -- feasible:
     { time: 0.051235, evals: 200, done: 1, feasible: 1, sos: 9.53751, f: 0, ineq: 0.0602805, eq: 0.340924 }
  -- feasible:
     { time: 0.079095, evals: 201, done: 1, feasible: 1, sos: 9.97874, f: 0, ineq: 0.0614884, eq: 0.340507 }
  -- feasible:
     { time: 0.0145, evals: 38, done: 1, feasible: 1, sos: 1.55816, f: 0, ineq: 6.18967e-07, eq: 1.93568e-06 }
  -- feasible:
     { time: 0.020767, evals: 70, done: 1, feasible: 1, sos: 9.02855, f: 0, ineq: 5.18334e-07, eq: 5.84549e-06 }
  -- feasible:
     { time: 0.032381, evals: 157, done: 1, feasible: 1, sos: 6.23748, f: 0, ineq: 0, eq: 0.0123802 }
  -- feasible:
     { time: 0.03224, evals: 173, done: 1, feasible: 1, sos: 20.3903, f: 0, ineq: 0.000316202, eq: 0.000378403 }
  -- feasible:
     { time: 0.043399, evals: 201, done: 1, feasible: 1, sos: 19.5482, f: 0, ineq: 0.00706512, eq: 0.00550651 

### Cylinder grasp

A cylinder (or capsule) can be grasped by ensuring the finger axis is normal to the cylinder's axis -- have a look at the `grasp_cylinder` method. Again, a demo with random bias to show the variety of grasps modelled that way:

In [5]:
C.setJointState(qHome)
limits = C.getJointLimits()
for i in range(10):
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
    man.bias(1., qBias, 1e0)
    man.grasp_cylinder(1., gripper, 'capsule', palm)
    
    # solve
    pose = man.solve()
    
    # if feasible, display
    if man.ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_cylinder\nret: {man.ret}')
    else:
        print('-- infeasible', i, orientation)

  -- feasible:
     { time: 0.012121, evals: 69, done: 1, feasible: 1, sos: 5.1126, f: 0, ineq: 0, eq: 0.00945172 }
  -- feasible:
     { time: 0.027136, evals: 128, done: 1, feasible: 1, sos: 21.7747, f: 0, ineq: 0, eq: 2.12634e-06 }
  -- feasible:
     { time: 0.013661, evals: 72, done: 1, feasible: 1, sos: 5.39811, f: 0, ineq: 0, eq: 0.00945185 }
  -- feasible:
     { time: 0.020003, evals: 73, done: 1, feasible: 1, sos: 5.38886, f: 0, ineq: 1.52415e-05, eq: 1.3364e-05 }
  -- feasible:
     { time: 0.025636, evals: 72, done: 1, feasible: 1, sos: 4.72131, f: 0, ineq: 0, eq: 0.00945157 }
  -- feasible:
     { time: 0.038793, evals: 201, done: 1, feasible: 1, sos: 3.60318, f: 0, ineq: 0, eq: 0.00257859 }
  -- feasible:
     { time: 0.020099, evals: 109, done: 1, feasible: 1, sos: 20.5991, f: 0, ineq: 0, eq: 2.29818e-06 }
  -- feasible:
     { time: 0.016809, evals: 54, done: 1, feasible: 1, sos: 3.07853, f: 0, ineq: 0, eq: 9.36283e-06 }
  -- feasible:
     { time: 0.027713, evals: 115,

## Sequential Manipulation Modelling

Sequential manipulation modelling is special, as in some phases the manipulated objects move with the manipulator. Internally, komo models this with a *mode switch* (where an object becomes attached to the manipulator with a stable (but optimizable) relative transform).

Using the ManipulationModelling class, the `setup_pick_and_place_waypoints` method creates a two-time-steps komo problem where the relative object-gripper position is constrained to be the same in the 1st and 2st step (as it is parameterized by a shared relative kinematic joint). The `grasp_box` method ensures that the solution *also* fulfils grasp constraints in the first time step; and the `place_box` method ensures that the solution *also* fulfils placement constraints in the second time step. The additional `target_relative_xy_position` is optional, so see placement to explicit xy-positions on the table. Have a look at the definitions of all these methods.

In [6]:
C.setJointState(qHome)

for i in range(10):
    grasp_ori = random.choice(['x', 'y', 'z'])
    place_ori = 'z' #random.choice(['x', 'y', 'z'])
    info = f'pnp {i}, grasp orientation {grasp_ori}, place orientation {place_ori}'
    print('===', info)
    
    # setup manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_pick_and_place_waypoints(gripper, box)
    man.grasp_box(1., gripper, box, palm, grasp_ori)
    man.place_box(2., box, table, palm, place_ori)
    man.target_relative_xy_position(2., box, table, [(i%5)*.1-.2, .2])
    
    # solve
    q = man.solve()

    # if feasible, display (including 'fake' simulation with kinematic attach)
    if man.ret.feasible:
        C.setJointState(q[0])
        C.view(True, f'{info}\nwaypoint 0\nret: {man.ret}')
        C.attach(gripper, box)
        C.setJointState(q[1])
        C.view(True, f'{info}\nwaypoint 1\nret: {man.ret}')
        C.attach(table, box)
        C.setJointState(qHome)
        C.view(True, 'back home')
    else:
        print(' -- infeasible')

del man
C.frame('box').setPosition([-.25,.1,1.])
C.view()

=== pnp 0, grasp orientation y, place orientation z
  -- feasible:
     { time: 0.031373, evals: 46, done: 1, feasible: 1, sos: 0.0219559, f: 0, ineq: 0, eq: 3.60174e-06 }
=== pnp 1, grasp orientation x, place orientation z
  -- feasible:
     { time: 0.05742, evals: 87, done: 1, feasible: 1, sos: 0.0411171, f: 0, ineq: 0.000237294, eq: 0.00281096 }
=== pnp 2, grasp orientation y, place orientation z
  -- feasible:
     { time: 0.038541, evals: 57, done: 1, feasible: 1, sos: 0.0225928, f: 0, ineq: 0, eq: 4.94079e-06 }
=== pnp 3, grasp orientation x, place orientation z
  -- feasible:
     { time: 0.04858, evals: 70, done: 1, feasible: 1, sos: 0.03018, f: 0, ineq: 3.36441e-08, eq: 3.86046e-06 }
=== pnp 4, grasp orientation y, place orientation z
  -- feasible:
     { time: 0.017345, evals: 26, done: 1, feasible: 1, sos: 0.0208696, f: 0, ineq: 8.02946e-10, eq: 1.73167e-06 }
=== pnp 5, grasp orientation z, place orientation z
  -- infeasible:
     { time: 0.149316, evals: 201, done: 1, fe

0

## Path generation

Once solutions to the manipulation keyframes/waypoints are available, the next step is to generate motion between them. We can use sample-based path finding (bi-directional RRT) and/or smooth motion optimization for this.

### Smooth point-to-point motion
The following demonstrates smooth point-to-point motion between box grasps, there the motion is additionally constrains the endeffector to retract and approach:

In [7]:
C.setJointState(qHome)
limits = C.getJointLimits()
verbose = 0

for i in range(20):
    qStart = C.getJointState()
    
    # choose a random grasp orientation
    orientation = random.choice(['x', 'y', 'z'])
    print('===', i, 'orientation', orientation)
    
    # setup the grasp problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics(accumulated_collisions=True)
    man.grasp_box(1., gripper, box, palm, orientation)
    
    # solve
    pose = man.solve()
    print('    IK:', man.ret)
    
    # if feasible, display; otherwise try another grasp
    if man.ret.feasible:
        if verbose>0:
            C.setJointState(pose[0])
            C.view(True, f'grasp {i} with orientation {orientation}\nret: {man.ret}')
    else:
        print('  -- infeasible')
        C.setJointState(qStart)
        continue

    # setup the motion problem
    man = manip.ManipulationModelling(C, helpers=[gripper])
    man.setup_point_to_point_motion(qStart, pose[0])
    man.retract([.0, .2], gripper)
    man.approach([.8, 1.], gripper)
    
    # solve
    path = man.solve()
    print('  path:', man.ret)

    # if feasible, display trivially (no real execution in BotOp here)
    if man.ret.feasible:
        for t in range(path.shape[0]):
            C.setJointState(path[t])
            C.view(False, f'grasp {i} with orientation {orientation}, path step {t}\n{man.ret}')
            time.sleep(.05)
        C.view(verbose>0, f'path done')
    else:
        print('  -- infeasible')
        
del man

=== 0 orientation y
  -- feasible:
     { time: 0.007456, evals: 33, done: 1, feasible: 1, sos: 0.0164863, f: 0, ineq: 4.00346e-08, eq: 2.87349e-07 }
    IK: { time: 0.007456, evals: 33, done: 1, feasible: 1, sos: 0.0164863, f: 0, ineq: 4.00346e-08, eq: 2.87349e-07 }
  -- feasible:
     { time: 0.023706, evals: 21, done: 1, feasible: 1, sos: 0.932886, f: 0, ineq: 7.91043e-07, eq: 9.19402e-05 }
  path: { time: 0.023706, evals: 21, done: 1, feasible: 1, sos: 0.932886, f: 0, ineq: 7.91043e-07, eq: 9.19402e-05 }
=== 1 orientation x
  -- feasible:
     { time: 0.016075, evals: 50, done: 1, feasible: 1, sos: 0.0562458, f: 0, ineq: 7.76546e-08, eq: 7.68614e-07 }
    IK: { time: 0.016075, evals: 50, done: 1, feasible: 1, sos: 0.0562458, f: 0, ineq: 7.76546e-08, eq: 7.68614e-07 }
  -- feasible:
     { time: 0.036401, evals: 34, done: 1, feasible: 1, sos: 1.11044, f: 0, ineq: 1.19127e-06, eq: 0.00066616 }
  path: { time: 0.036401, evals: 34, done: 1, feasible: 1, sos: 1.11044, f: 0, ineq: 1.1912

## Integrated Example

Let's start with an integrated example, where the robot endlessly loops through picking and placing a box on a table.

In [8]:
import robotic as ry
import manipulation as manip
import numpy as np
#from importlib import reload
import time
import random

We define a basic configuration with box on the table:

In [9]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.addFrame("box", "table") \
    .setJoint(ry.JT.rigid) \
    .setShape(ry.ST.ssBox, [.15,.06,.06,.005]) \
    .setRelativePosition([-.0,.3-.055,.095]) \
    .setContact(1) \
    .setMass(.1)

C.addFrame("obstacle", "table") \
    .setShape(ry.ST.ssBox, [.06,.15,.06,.005]) \
    .setColor([.1]) \
    .setRelativePosition([-.15,.3-.055,.095]) \
    .setContact(1)

C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

### endless box pick and place with random pick and place orientations

In [10]:
#reload(manip)

C.setJointState(qHome)
C.view_raise()

for l in range(20):
        qStart = C.getJointState()

        graspDirection = random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.no_collision([], palm, table)
        M.target_relative_xy_position(2., box, table, [.2, .3])
        ways = M.solve()

        if not M.feasible:
            continue

        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.no_collision([.3,.7], palm, box, margin=.05)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.no_collision([], table, box)
        M3.solve()
        if not M3.ret.feasible:
            continue

        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp z place z
  -- infeasible:placement 0: grasp z place z
     { time: 0.241409, evals: 200, done: 1, feasible: 0, sos: 0.390688, f: 0, ineq: 0.253242, eq: 1.48787 }
=== placement 1: grasp z place z
  -- infeasible:placement 1: grasp z place z
     { time: 0.160418, evals: 200, done: 1, feasible: 0, sos: 0.23797, f: 0, ineq: 0.263042, eq: 1.52741 }
=== placement 2: grasp y place z
  -- feasible:placement 2: grasp y place z
     { time: 0.0348, evals: 53, done: 1, feasible: 1, sos: 0.0529068, f: 0, ineq: 0, eq: 1.86648e-07 }
  -- feasible:sub_motion_0--placement 2: grasp y place z
     { time: 0.041684, evals: 33, done: 1, feasible: 1, sos: 0.969778, f: 0, ineq: 5.63453e-07, eq: 6.65802e-05 }
  -- feasible:sub_motion_1--placement 2: grasp y place z
     { time: 0.012339, evals: 10, done: 1, feasible: 1, sos: 0.0413613, f: 0, ineq: 0.000265603, eq: 0.00294508 }
=== placement 3: grasp z place y
  -- infeasible:placement 3: grasp z place y
     { time: 0.143251, evals: 

### box top grasps and place over obstacle

In [11]:
#reload(manip)

C.setJointState(qHome)
C.view_raise()

C.frame(box).setRelativePosition([-.0,.3-.055,.095])
C.frame(box).setRelativeQuaternion([1.,0,0,0])

for i in range(7):
        qStart = C.getJointState()

        graspDirection = 'yz' #random.choice(['xz', 'yz'])
        placeDirection = 'z'
        place_position = [(i%3)*.3-.3, .2]
        place_orientation = [-(i%2),((i+1)%2),0.]
        info = f'placement {i}: grasp {graspDirection} place {placeDirection} place_pos {place_position} place_ori {place_orientation}'
        print('===', info)

        M = manip.ManipulationModelling(C, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, joint_limits=False)
        M.grasp_top_box(1., gripper, box, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.target_relative_xy_position(2., box, table, place_position)
        M.target_x_orientation(2., box, place_orientation)
        M.solve()
        if not M.feasible:
                continue

        M2 = M.sub_motion(0)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.ret.feasible:
            continue

        M3 = M.sub_motion(1)
#         M3.retract([.0, .2], box, distance=.05)
#         M3.approach([.8, 1.], box, distance=.05)
        M3.no_collision([], table, box)
        M3.no_collision([], box, 'obstacle')
        M3.bias(.5, qHome, 1e0)
        M3.solve()
        if not M3.ret.feasible:
            continue
            
        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp yz place z place_pos [-0.3, 0.2] place_ori [0, 1, 0.0]
  -- feasible:
     { time: 0.013284, evals: 16, done: 1, feasible: 1, sos: 0.0739063, f: 0, ineq: 0, eq: 0.000157831 }
  -- feasible:sub_motion_0--
     { time: 0.040566, evals: 28, done: 1, feasible: 1, sos: 1.02374, f: 0, ineq: 5.78372e-07, eq: 5.9725e-05 }
  -- feasible:sub_motion_1--
     { time: 0.013717, evals: 9, done: 1, feasible: 1, sos: 1.53591, f: 0, ineq: 0.000494294, eq: 0.00365479 }
=== placement 1: grasp yz place z place_pos [0.0, 0.2] place_ori [-1, 0, 0.0]
  -- feasible:
     { time: 0.117973, evals: 200, done: 1, feasible: 1, sos: 0.235816, f: 0, ineq: 0, eq: 0.115075 }
  -- feasible:sub_motion_0--
     { time: 0.035168, evals: 29, done: 1, feasible: 1, sos: 0.966664, f: 0, ineq: 2.60606e-06, eq: 0.000457442 }
  -- feasible:sub_motion_1--
     { time: 0.11104, evals: 73, done: 1, feasible: 1, sos: 5.60099, f: 0, ineq: 0.00263072, eq: 0.00782843 }
=== placement 2: grasp yz place z place_pos 

In [12]:
#reload(manip)

C.frame("l_panda_finger_joint1").setJointState(np.array([.0]))

obj = box
C.frame(obj).setRelativePosition([-.0,.3-.055,.095])
C.frame(obj).setRelativeQuaternion([1.,0,0,0])

for i in range(50):
     qStart = C.getJointState()

     info = f'push {i}'
     print('===', info)

     M = manip.ManipulationModelling(C, info, ['l_gripper'])
     M.setup_pick_and_place_waypoints(gripper, obj, 1e-1)
     M.straight_push([1.,2.], obj, gripper, table)
     #random target position
     M.komo.addObjective([2.], ry.FS.position, [obj], ry.OT.eq, 1e1*np.array([[1,0,0],[0,1,0]]), .4*np.random.rand(3) - .2+np.array([.0,.3,.0]))
     M.solve()
     if not M.ret.feasible:
          continue

     M1 = M.sub_motion(0)
     M1.retractPush([.0, .15], gripper, .03)
     M1.approachPush([.85, 1.], gripper, .03)
     M1.no_collision([.15,.85], obj, "l_finger1", .02)
     M1.no_collision([.15,.85], obj, "l_finger2", .02)
     M1.no_collision([.15,.85], obj, 'l_palm', .02)
     M1.no_collision([], table, "l_finger1", .0)
     M1.no_collision([], table, "l_finger2", .0)
     M1.solve()
     if not M1.ret.feasible:
          continue

     M2 = M.sub_motion(1)
     M2.komo.addObjective([], ry.FS.positionRel, [gripper, '_push_start'], ry.OT.eq, 1e1*np.array([[1,0,0],[0,0,1]]))
     M2.solve()
     if not M2.ret.feasible:
          continue

     M1.play(C, 1.)
     C.attach(gripper, obj)
     M2.play(C, 1.)
     C.attach(table, obj)


=== push 0
  -- feasible:push 0
     { time: 0.048366, evals: 70, done: 1, feasible: 1, sos: 0.316815, f: 0, ineq: 4.03625e-05, eq: 0.0353098 }
  -- feasible:sub_motion_0--push 0
     { time: 0.072406, evals: 31, done: 1, feasible: 1, sos: 2.39085, f: 0, ineq: 3.78529e-07, eq: 0.000107627 }
  -- feasible:sub_motion_1--push 0
     { time: 0.016443, evals: 11, done: 1, feasible: 1, sos: 0.0967844, f: 0, ineq: 0, eq: 0.00127421 }
=== push 1
  -- feasible:push 1
     { time: 0.02048, evals: 39, done: 1, feasible: 1, sos: 0.124928, f: 0, ineq: 0, eq: 0.084521 }
  -- feasible:sub_motion_0--push 1
     { time: 0.44944, evals: 201, done: 1, feasible: 1, sos: 1.4798, f: 0, ineq: 0.00252944, eq: 0.000127623 }
  -- feasible:sub_motion_1--push 1
     { time: 0.011172, evals: 8, done: 1, feasible: 1, sos: 0.0181492, f: 0, ineq: 0, eq: 0.0187794 }
=== push 2
  -- feasible:push 2
     { time: 0.008888, evals: 16, done: 1, feasible: 1, sos: 0.0430163, f: 0, ineq: 0, eq: 0.0345614 }
  -- feasible:sub_m

## TODOS:
* Proper execution: BotOp instead of display with C
* RRTs
* additional planar motion constraint for in-plane manipulation
* more typical manipulation constraints: camera_look_at, push_straight, 

In [13]:
del C